import everything

In [148]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

plays = pd.read_csv(r'C:\Users\18172\Desktop\Big data bowl\plays.csv')
week1 = pd.read_csv(r'C:\Users\18172\Desktop\Big data bowl\week1.csv')
pff = pd.read_csv(r'C:\Users\18172\Desktop\Big data bowl\pffScoutingData.csv')
players = pd.read_csv(r'C:\Users\18172\Desktop\Big data bowl\players.csv')

next two cells are copied from a different notebook, can adjust nums later to focus in on olineman, good for testing our methods for detecting diff plays

In [149]:
colors = {
    'ARI':"#97233F", 
    'ATL':"#A71930", 
    'BAL':'#241773', 
    'BUF':"#00338D", 
    'CAR':"#0085CA", 
    'CHI':"#C83803", 
    'CIN':"#FB4F14", 
    'CLE':"#311D00", 
    'DAL':'#003594',
    'DEN':"#FB4F14", 
    'DET':"#0076B6", 
    'GB':"#203731", 
    'HOU':"#03202F", 
    'IND':"#002C5F", 
    'JAX':"#9F792C", 
    'KC':"#E31837", 
    'LA':"#003594", 
    'LAC':"#0080C6", 
    'LV':"#000000",
    'MIA':"#008E97", 
    'MIN':"#4F2683", 
    'NE':"#002244", 
    'NO':"#D3BC8D", 
    'NYG':"#0B2265", 
    'NYJ':"#125740", 
    'PHI':"#004C54", 
    'PIT':"#FFB612", 
    'SEA':"#69BE28", 
    'SF':"#AA0000",
    'TB':'#D50A0A', 
    'TEN':"#4B92DB", 
    'WAS':"#5A1414", 
    'football':'#CBB67C'
}


In [150]:
def animate_play(tracking_df, play_df,players,pffScoutingData, gameId,playId):
    selected_play_df = play_df[(play_df.playId==playId)&(play_df.gameId==gameId)].copy()
    
    tracking_players_df = pd.merge(tracking_df,players,how="left",on = "nflId")
    tracking_players_df = pd.merge(tracking_players_df,pffScoutingData,how="left",on = ["nflId","playId","gameId"])
    selected_tracking_df = tracking_players_df[(tracking_players_df.playId==playId)&(tracking_players_df.gameId==gameId)].copy()

    sorted_frame_list = selected_tracking_df.frameId.unique()
    sorted_frame_list.sort()

    # get play General information 
    line_of_scrimmage = selected_play_df.absoluteYardlineNumber.values[0]
    first_down_marker = line_of_scrimmage + selected_play_df.yardsToGo.values[0]
    down = selected_play_df.down.values[0]
    quarter = selected_play_df.quarter.values[0]
    gameClock = selected_play_df.gameClock.values[0]
    playDescription = selected_play_df.playDescription.values[0]
    # Handle case where we have a really long Play Description and want to split it into two lines
    if len(playDescription.split(" "))>15 and len(playDescription)>115:
        playDescription = " ".join(playDescription.split(" ")[0:16]) + "<br>" + " ".join(playDescription.split(" ")[16:])

    # initialize plotly start and stop buttons for animation
    updatemenus_dict = [
        {
            "buttons": [
                {
                    "args": [None, {"frame": {"duration": 100, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 0}}],
                    "label": "Play",
                    "method": "animate"
                },
                {
                    "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                      "mode": "immediate",
                                      "transition": {"duration": 0}}],
                    "label": "Pause",
                    "method": "animate"
                }
            ],
            "direction": "left",
            "pad": {"r": 10, "t": 87},
            "showactive": False,
            "type": "buttons",
            "x": 0.1,
            "xanchor": "right",
            "y": 0,
            "yanchor": "top"
        }
    ]
    # initialize plotly slider to show frame position in animation
    sliders_dict = {
        "active": 0,
        "yanchor": "top",
        "xanchor": "left",
        "currentvalue": {
            "font": {"size": 20},
            "prefix": "Frame:",
            "visible": True,
            "xanchor": "right"
        },
        "transition": {"duration": 300, "easing": "cubic-in-out"},
        "pad": {"b": 10, "t": 50},
        "len": 0.9,
        "x": 0.1,
        "y": 0,
        "steps": []
    }


    frames = []
    for frameId in sorted_frame_list:
        data = []
        # Add Numbers to Field 
        data.append(
            go.Scatter(
                x=np.arange(20,110,10), 
                y=[5]*len(np.arange(20,110,10)),
                mode='text',
                text=list(map(str,list(np.arange(20, 61, 10)-10)+list(np.arange(40, 9, -10)))),
                textfont_size = 30,
                textfont_family = "Courier New, monospace",
                textfont_color = "#ffffff",
                showlegend=False,
                hoverinfo='none'
            )
        )
        data.append(
            go.Scatter(
                x=np.arange(20,110,10), 
                y=[53.5-5]*len(np.arange(20,110,10)),
                mode='text',
                text=list(map(str,list(np.arange(20, 61, 10)-10)+list(np.arange(40, 9, -10)))),
                textfont_size = 30,
                textfont_family = "Courier New, monospace",
                textfont_color = "#ffffff",
                showlegend=False,
                hoverinfo='none'
            )
        )
        # Add line of scrimage 
        data.append(
            go.Scatter(
                x=[line_of_scrimmage,line_of_scrimmage], 
                y=[0,53.5],
                line_dash='dash',
                line_color='blue',
                showlegend=False,
                hoverinfo='none'
            )
        )
        # Add First down line 
        data.append(
            go.Scatter(
                x=[first_down_marker,first_down_marker], 
                y=[0,53.5],
                line_dash='dash',
                line_color='yellow',
                showlegend=False,
                hoverinfo='none'
            )
        )
        # Plot Players
        for team in selected_tracking_df.team.unique():
            plot_df = selected_tracking_df[(selected_tracking_df.team==team)&(selected_tracking_df.frameId==frameId)].copy()
            if team != "football":
                hover_text_array=[]
                for nflId in plot_df.nflId:
                    selected_player_df = plot_df[plot_df.nflId==nflId]
                    hover_text_array.append("nflId:{}<br>displayName:{}<br>Position:{}<br>Role:{}".format(selected_player_df["nflId"].values[0],
                                                                                      selected_player_df["displayName"].values[0],
                                                                                      selected_player_df["pff_positionLinedUp"].values[0],
                                                                                      selected_player_df["pff_role"].values[0]))
                data.append(go.Scatter(x=plot_df["x"], y=plot_df["y"],mode = 'markers',marker_color=colors[team],name=team,hovertext=hover_text_array,hoverinfo="text"))
            else:
                data.append(go.Scatter(x=plot_df["x"], y=plot_df["y"],mode = 'markers',marker_color=colors[team],name=team,hoverinfo='none'))
        # add frame to slider
        slider_step = {"args": [
            [frameId],
            {"frame": {"duration": 100, "redraw": False},
             "mode": "immediate",
             "transition": {"duration": 0}}
        ],
            "label": str(frameId),
            "method": "animate"}
        sliders_dict["steps"].append(slider_step)
        frames.append(go.Frame(data=data, name=str(frameId)))

    scale=10
    layout = go.Layout(
        autosize=False,
        width=120*scale,
        height=60*scale,
        xaxis=dict(range=[0, 120], autorange=False, tickmode='array',tickvals=np.arange(10, 111, 5).tolist(),showticklabels=False),
        yaxis=dict(range=[0, 53.3], autorange=False,showgrid=False,showticklabels=False),

        plot_bgcolor='#00B140',
        # Create title and add play description at the bottom of the chart for better visual appeal
        title=f"GameId: {gameId}, PlayId: {playId}<br>{gameClock} {quarter}Q"+"<br>"*19+f"{playDescription}",
        updatemenus=updatemenus_dict,
        sliders = [sliders_dict]
    )

    fig = go.Figure(
        data=frames[0]["data"],
        layout= layout,
        frames=frames[1:]
    )
    # Create First Down Markers 
    for y_val in [0,53]:
        fig.add_annotation(
                x=first_down_marker,
                y=y_val,
                text=str(down),
                showarrow=False,
                font=dict(
                    family="Courier New, monospace",
                    size=16,
                    color="black"
                    ),
                align="center",
                bordercolor="black",
                borderwidth=2,
                borderpad=4,
                bgcolor="#ff7f0e",
                opacity=1
                )

    return fig

In [153]:
animate_play(week1,plays,players,pff,2021091203,672).show()

A few things going on here, I create a unique id to be able to look at specific plays within games, I merge tracking data with the pff info, I subset the first 2 seconds of every play, I also create new columns for the line of scrimmage and point of the ball, last I subset every pff pass rush role in the tracking data

In [154]:
data = week1.merge(pff, how = 'left')

data['uId'] = data['gameId'].to_string() + "-" + data['playId'].to_string()

idxs = (data
        .loc[data['event'] == 'ball_snap',
                 'frameId']
        .index
        .values)

x = [(idxs+x).tolist() for x in range(0,24)]
idxs = [item for sublist in x for item in sublist]

df = data.loc[idxs]

_los = (data
        .loc[(data['team']=='football') &
             (data['frameId']==1),
             ['gameId', 'playId', 'x']]
        .rename(columns={'x':'los'}))

_ball = (data
        .loc[(data['team']=='football') &
             (data['frameId']==1),
             ['gameId', 'playId', 'y']]
        .rename(columns={'y':'cop'}))

df = df.merge(_los)
df = df.merge(_ball)

p_r = df[df['pff_role'] == "Pass Rush"]

this is just to check to see what an average play setup looks like, one thing I have learned is there is so much data you can't just upload to a csv or use a variable finder on an ide bc it just crashes, have to subset everything

In [155]:
p_r.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119706 entries, 6 to 646942
Data columns (total 31 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   gameId                  119706 non-null  int64  
 1   playId                  119706 non-null  int64  
 2   nflId                   119706 non-null  float64
 3   frameId                 119706 non-null  int64  
 4   time                    119706 non-null  object 
 5   jerseyNumber            119706 non-null  float64
 6   team                    119706 non-null  object 
 7   playDirection           119706 non-null  object 
 8   x                       119706 non-null  float64
 9   y                       119706 non-null  float64
 10  s                       119706 non-null  float64
 11  a                       119706 non-null  float64
 12  dis                     119706 non-null  float64
 13  o                       119706 non-null  float64
 14  dir                 

this function sets up zones along the line of scrimmage with a players position, I think this is probably easier to work with than just using a players y coordinate, especially when comparing between players

In [156]:
def zones (row):
    rel_pos = row['cop'] - row['y']
    
    #add a line that accounts for non lineman, and sets their zone to something different
    #also need to determine what direction the defense is facing and adjust accordingly
    #rn I have 14 zones at half a yard each which is arbitrary
    pos = row["pff_positionLinedUp"] 
    corners = ["LCB", "RCB", "SCBiL", "SCBiR", "SCBL",  "SCBoL", "SCBoR", "SCBR"]
#     if pos in corners:
#         if row['playDirection'] == 'right':
#             if rel_pos<-3:
#                 return -100
#             else:
#                 return 100
#         if row['playDirection'] == 'left':
#             if rel_pos<-3:
#                 return 100
#             else:
#                 return -100
    
#     vert_pos = row['los'] - row['y']
    
#     if abs(vert_pos > 1.5):
#         return -2
    
    if row['playDirection'] == 'right':
        if (rel_pos < -3):
            return 1
        elif (rel_pos < -2.5):
            return 2
        elif (rel_pos < -2):
            return 3
        elif (rel_pos < -1.5):
            return 4
        elif (rel_pos < -1):
            return 5
        elif (rel_pos < -.5):
            return 6
        elif (rel_pos < 0):
            return 7
        elif (rel_pos < .5):
            return 8
        elif (rel_pos < 1):
            return 9
        elif (rel_pos < 1.5):
            return 10
        elif (rel_pos < 2):
            return 11
        elif (rel_pos < 2.5):
            return 12
        elif (rel_pos < 3):
            return 13
        elif (rel_pos >= 3):
            return 14
        return 'Other'
    else:
        if (rel_pos < -3):
            return 14
        elif (rel_pos < -2.5):
            return 13
        elif (rel_pos < -2):
            return 12
        elif (rel_pos < -1.5):
            return 1
        elif (rel_pos < -1):
            return 10
        elif (rel_pos < -.5):
            return 9
        elif (rel_pos < 0):
            return 8
        elif (rel_pos < .5):
            return 7
        elif (rel_pos < 1):
            return 6
        elif (rel_pos < 1.5):
            return 5
        elif (rel_pos < 2):
            return 4
        elif (rel_pos < 2.5):
            return 3
        elif (rel_pos < 3):
            return 2
        elif (rel_pos >= 3):
            return 1
        return 'Other'

In [157]:
p_r['zone'] = p_r.apply (lambda row: zones(row), axis=1)

C:\Users\18172\AppData\Local\Temp\ipykernel_32552\2634120852.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



next step is to start figuring out plays, will have to have a column with an indicator to figure out when a player is in front of another player

In [158]:
def posdef (row):
    pos = row['pff_positionLinedUp']
    corners = ["LCB", "RCB", "SCBiL", "SCBiR", "SCBL",  "SCBoL", "SCBoR", "SCBR"]
    vert_pos = (row['los'] - row['x'])
    if row['playDirection'] == "right":
        vert_pos = vert_pos * -1
        
    if pos in corners:
        return(3)        
    elif vert_pos>1.5:
        return(2)
    else:
        return(1)

In [159]:
p_r['npos'] = p_r.apply (lambda row: posdef(row), axis=1)

C:\Users\18172\AppData\Local\Temp\ipykernel_32552\1745411953.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [160]:
current_timestamp = None
locations_at_timestamp = []
relative_locations_at_timestamp = {}

for index, row in p_r.iterrows():
    if row['time'] != current_timestamp:
        # New timestamp, so update the current timestamp and reset the list of locations
        current_timestamp = row['time']
        locations_at_timestamp = []
    pos = row["pff_positionLinedUp"] 
    corners = ["LCB", "RCB", "SCBiL", "SCBiR", "SCBL",  "SCBoL", "SCBoR", "SCBR"]
    vert_pos = (row['los'] - row['x'])
    if row['playDirection'] == "right":
        vert_pos = vert_pos * -1
    

    locations_at_timestamp.append(row['y'])
    
    # Step 3: Sort the list of locations at each timestamp
    locations_at_timestamp.sort()
    
    # Step 4: Assign a relative location to each player at each timestamp
    relative_locations = {}
    for i, location in enumerate(locations_at_timestamp):
        relative_locations[location] = i + 1
        
    # Store the relative locations for each player at this timestamp
    relative_locations_at_timestamp[current_timestamp] = relative_locations



In [161]:
p_r.iloc[54097]

gameId                                                           2021091205
playId                                                                 2795
nflId                                                               53624.0
frameId                                                                  19
time                                                2021-09-12T19:15:24.700
jerseyNumber                                                           91.0
team                                                                    HOU
playDirection                                                         right
x                                                                     20.02
y                                                                     24.08
s                                                                      2.06
a                                                                      2.15
dis                                                                    0.22
o           

In [162]:
len(relative_locations_at_timestamp)

26991

In [163]:
time = "2021-09-12T17:07:40.400"
relative_locations_at_timestamp[time]

{24.56: 1, 26.77: 2, 30.78: 3, 33.14: 4}

In [164]:
counte = 0
count = 0
for index, row in p_r.iterrows():
    try:
        p_r.at[index, 'relative_location'] = relative_locations_at_timestamp[row['time']][row['y']]
        count += 1
    except KeyError:
        print("KeyError occurred at index {}".format(index))
        p_r.at[index, 'relative_location'] = 0
        counte += 1
        


C:\Users\18172\AppData\Local\Temp\ipykernel_32552\2874851222.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



KeyError occurred at index 54097
KeyError occurred at index 54098
KeyError occurred at index 54106
KeyError occurred at index 54107
KeyError occurred at index 54120
KeyError occurred at index 54121
KeyError occurred at index 54129
KeyError occurred at index 54130
KeyError occurred at index 54143
KeyError occurred at index 54144
KeyError occurred at index 54153
KeyError occurred at index 54166
KeyError occurred at index 54167
KeyError occurred at index 54175
KeyError occurred at index 54176
KeyError occurred at index 54189
KeyError occurred at index 54190
KeyError occurred at index 54198
KeyError occurred at index 54199
KeyError occurred at index 54212
KeyError occurred at index 54213
KeyError occurred at index 54221
KeyError occurred at index 54222
KeyError occurred at index 54235
KeyError occurred at index 54236
KeyError occurred at index 54244
KeyError occurred at index 54245
KeyError occurred at index 54258
KeyError occurred at index 54259
KeyError occurred at index 54267
KeyError o

In [165]:
print(counte)
print(count)

5005
114701


In [166]:
p_r.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119706 entries, 6 to 646942
Data columns (total 34 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   gameId                  119706 non-null  int64  
 1   playId                  119706 non-null  int64  
 2   nflId                   119706 non-null  float64
 3   frameId                 119706 non-null  int64  
 4   time                    119706 non-null  object 
 5   jerseyNumber            119706 non-null  float64
 6   team                    119706 non-null  object 
 7   playDirection           119706 non-null  object 
 8   x                       119706 non-null  float64
 9   y                       119706 non-null  float64
 10  s                       119706 non-null  float64
 11  a                       119706 non-null  float64
 12  dis                     119706 non-null  float64
 13  o                       119706 non-null  float64
 14  dir                 

In [167]:
def group_and_create_column(df, group_col, value_col):
  # Group the dataframe by the specified column
  grouped_df = df.groupby(group_col)
  
  # Create a new column with a list of all the values in the specified column for each group
  new_df = grouped_df[value_col].apply(list).reset_index()
  
  # Rename the column to include the name of the original column
  new_df = new_df.rename(columns={value_col: f"{value_col}_list"})
  
  return new_df

In [168]:
b = group_and_create_column(p_r, 'time', 'relative_location')

In [169]:
b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26991 entries, 0 to 26990
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   time                    26991 non-null  object
 1   relative_location_list  26991 non-null  object
dtypes: object(2)
memory usage: 421.9+ KB


In [170]:
m = pd.merge(p_r, b, on = "time", how = "outer")

In [171]:
m

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,...,pff_nflIdBlockedPlayer,pff_blockType,pff_backFieldBlock,uId,los,cop,zone,npos,relative_location,relative_location_list
0,2021090900,97,41263.0,6,2021-09-10T00:26:31.600,90.0,DAL,right,43.30,18.89,...,NaN,NaN,NaN,0 2021090900\n1 2021090900\n...,42.92,24.04,14,1,1.0,"[1.0, 5.0, 3.0, 2.0, 4.0]"
1,2021090900,97,42403.0,6,2021-09-10T00:26:31.600,94.0,DAL,right,43.90,32.63,...,NaN,NaN,NaN,0 2021090900\n1 2021090900\n...,42.92,24.04,1,1,5.0,"[1.0, 5.0, 3.0, 2.0, 4.0]"
2,2021090900,97,44955.0,6,2021-09-10T00:26:31.600,91.0,DAL,right,43.35,25.18,...,NaN,NaN,NaN,0 2021090900\n1 2021090900\n...,42.92,24.04,5,1,3.0,"[1.0, 5.0, 3.0, 2.0, 4.0]"
3,2021090900,97,53441.0,6,2021-09-10T00:26:31.600,11.0,DAL,right,43.68,21.93,...,NaN,NaN,NaN,0 2021090900\n1 2021090900\n...,42.92,24.04,12,1,2.0,"[1.0, 5.0, 3.0, 2.0, 4.0]"
4,2021090900,97,53504.0,6,2021-09-10T00:26:31.600,97.0,DAL,right,43.70,26.67,...,NaN,NaN,NaN,0 2021090900\n1 2021090900\n...,42.92,24.04,2,1,4.0,"[1.0, 5.0, 3.0, 2.0, 4.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119701,2021091300,4845,37240.0,30,2021-09-14T03:54:20.600,90.0,BAL,left,44.60,22.88,...,NaN,NaN,NaN,0 2021090900\n1 2021090900\n...,40.67,23.87,6,1,2.0,"[3.0, 2.0, 5.0, 6.0, 1.0, 4.0]"
119702,2021091300,4845,40042.0,30,2021-09-14T03:54:20.600,98.0,BAL,left,45.46,25.07,...,NaN,NaN,NaN,0 2021090900\n1 2021090900\n...,40.67,23.87,10,1,5.0,"[3.0, 2.0, 5.0, 6.0, 1.0, 4.0]"
119703,2021091300,4845,46259.0,30,2021-09-14T03:54:20.600,32.0,BAL,left,50.17,27.19,...,NaN,NaN,NaN,0 2021090900\n1 2021090900\n...,40.67,23.87,14,1,6.0,"[3.0, 2.0, 5.0, 6.0, 1.0, 4.0]"
119704,2021091300,4845,52506.0,30,2021-09-14T03:54:20.600,40.0,BAL,left,48.51,20.37,...,NaN,NaN,NaN,0 2021090900\n1 2021090900\n...,40.67,23.87,1,3,1.0,"[3.0, 2.0, 5.0, 6.0, 1.0, 4.0]"


In [172]:
m.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119706 entries, 0 to 119705
Data columns (total 35 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   gameId                  119706 non-null  int64  
 1   playId                  119706 non-null  int64  
 2   nflId                   119706 non-null  float64
 3   frameId                 119706 non-null  int64  
 4   time                    119706 non-null  object 
 5   jerseyNumber            119706 non-null  float64
 6   team                    119706 non-null  object 
 7   playDirection           119706 non-null  object 
 8   x                       119706 non-null  float64
 9   y                       119706 non-null  float64
 10  s                       119706 non-null  float64
 11  a                       119706 non-null  float64
 12  dis                     119706 non-null  float64
 13  o                       119706 non-null  float64
 14  dir                 

In [177]:
z = m[['gameId', 'playId', 'nflId', 'time', 'team', 'playDirection', 'x', 'y', 'event', 'pff_role', 'pff_positionLinedUp', 'los', 'cop', 'zone', 'relative_location', 'relative_location_list', 'npos']]

In [178]:
z.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119706 entries, 0 to 119705
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   gameId                  119706 non-null  int64  
 1   playId                  119706 non-null  int64  
 2   nflId                   119706 non-null  float64
 3   time                    119706 non-null  object 
 4   team                    119706 non-null  object 
 5   playDirection           119706 non-null  object 
 6   x                       119706 non-null  float64
 7   y                       119706 non-null  float64
 8   event                   119706 non-null  object 
 9   pff_role                119706 non-null  object 
 10  pff_positionLinedUp     119706 non-null  object 
 11  los                     119706 non-null  float64
 12  cop                     119706 non-null  float64
 13  zone                    119706 non-null  int64  
 14  relative_location   

In [189]:
z.iloc[0:50]

,gameId,playId,nflId,time,team,playDirection,x,y,event,pff_role,pff_positionLinedUp,los,cop,zone,relative_location,relative_location_list,npos
0,2021090900,97,41263.0,2021-09-10T00:26:31.600,DAL,right,43.30,18.89,ball_snap,Pass Rush,LEO,42.92,24.04,14,1.0,"[1.0, 5.0, 3.0, 2.0, 4.0]",1
1,2021090900,97,42403.0,2021-09-10T00:26:31.600,DAL,right,43.90,32.63,ball_snap,Pass Rush,ROLB,42.92,24.04,1,5.0,"[1.0, 5.0, 3.0, 2.0, 4.0]",1
2,2021090900,97,44955.0,2021-09-10T00:26:31.600,DAL,right,43.35,25.18,ball_snap,Pass Rush,DRT,42.92,24.04,5,3.0,"[1.0, 5.0, 3.0, 2.0, 4.0]",1
3,2021090900,97,53441.0,2021-09-10T00:26:31.600,DAL,right,43.68,21.93,ball_snap,Pass Rush,LILB,42.92,24.04,12,2.0,"[1.0, 5.0, 3.0, 2.0, 4.0]",1
4,2021090900,97,53504.0,2021-09-10T00:26:31.600,DAL,right,43.70,26.67,ball_snap,Pass Rush,RE,42.92,24.04,2,4.0,"[1.0, 5.0, 3.0, 2.0, 4.0]",1
5,2021090900,97,41263.0,2021-09-10T00:26:31.700,DAL,right,43.19,18.91,None,Pass Rush,LEO,42.92,24.04,14,1.0,"[1.0, 5.0, 3.0, 2.0, 4.0]",1
6,2021090900,97,42403.0,2021-09-10T00:26:31.700,DAL,right,43.83,32.62,None,Pass Rush,ROLB,42.92,24.04,1,5.0,"[1.0, 5.0, 3.0, 2.0, 4.0]",1
7,2021090900,97,44955.0,2021-09-10T00:26:31.700,DAL,right,43.28,25.20,None,Pass Rush,DRT,42.92,24.04,5,3.0,"[1.0, 5.0, 3.0, 2.0, 4.0]",1
8,2021090900,97,53441.0,2021-09-10T00:26:31.700,DAL,right,43.62,21.96,None,Pass Rush,LILB,42.92,24.04,12,2.0,"[1.0, 5.0, 3.0, 2.0, 4.0]",1
9,2021090900,97,53504.0,2021-09-10T00:26:31.700,DAL,right,43.67,26.67,None,Pass Rush,RE,42.92,24.04,2,4.0,"[1.0, 5.0, 3.0, 2.0, 4.0]",1
